## Creates a .csv file with the landmarks coordinates obtained from images or videos

In [1]:
#Import requirements libraries

import cv2
import mediapipe as mp
import os
import pathlib
import csv
import numpy as np
from glob import iglob

In [2]:
class GetCoordsLandmarks():

    '''class to obtain the coordinates of the landmarks form body images or videos using MediaPipe-Pose library'''
    
    def __init__(self):
                
        self.file_list_used= []
        self.mp_pose = mp.solutions.pose # Mediapipe Solutions
        pass
    
    def create_csv_file(self, file_name: str):

        '''"file_name": The name of the csv file as it will be stored. ("file_name.csv").'''

        self.file_name = file_name + '.csv'
       
        landmarks = ['class']
        for val in range(1, 34):
            landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
        
        with open(file_name + '.csv', mode='w', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(landmarks) 

    
    def get_coord(self, rootdir: str, class_name = None, file_name = None, min_detection_confidence=0.6, min_tracking_confidence=0.6):

        '''
        Create a dataset with the coord of the landmarks of the images or videos stored in "rootdir"

        "rootdir": Path of the folder where the images or videos are stored.

        "class_name" : str, default = None - If not given, will be the folder name where the images or video are stored.

        "file_name" : str, default = None - If not given, the file created in "create_csv_file()" will be used.

        "min_detection_confidence" : mp_pose min_detection_confidence (default = 0.6).

        "min_tracking_confidence": mp_pose min_detection_confidence (default = 0.6).
        
        '''

        self.rootdir_glob = rootdir

        if file_name == None:
            file_name = self.file_name

        path_files_list = [f for f in iglob(rootdir, recursive=True) if os.path.isfile(f)]
    
        for path in path_files_list:

            if class_name == None:        
                class_name = pathlib.PurePath(os.path.normpath(path))
                class_name = class_name.parent.name
            
            cap = cv2.VideoCapture(path)

            with self.mp_pose.Pose(min_detection_confidence=min_detection_confidence, min_tracking_confidence=min_tracking_confidence) as pose:
                
                while cap.isOpened():
                    ret, frame = cap.read()

                    if frame is None:
                        break
                    
                    # Recolor Feed
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    image.flags.writeable = False        
                    
                    # Make Detections
                    results = pose.process(image)
                   
                    # Recolor image back to BGR for rendering
                    image.flags.writeable = True   
                    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    
                    # Export coordinates
                    try:
                        # Extract Pose landmarks
                        poses = results.pose_landmarks.landmark
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in poses]).flatten())
                      
                        # Append class name
                        pose_row.insert(0, class_name)
                        self.file_list_used.append(path)
                        
                        # Export to CSV
                        with open(file_name, mode='a', newline='') as f:
                            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                            csv_writer.writerow(pose_row) 
                        
                    except:
                        pass
                                    
                    #cv2.imshow('Video', image)

                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break

            cap.release()
            cv2.destroyAllWindows()
        
    def bad_images(self):

        '''Return the images that are bad for detection'''
    
        path_files_list = [f for f in iglob(self.rootdir_glob, recursive=True) if os.path.isfile(f)]
        

        images_not_used=[]
        for j in path_files_list:
            if j not in self.file_list_used:
                images_not_used.append(j)
        
        print(f'{len(images_not_used)} bad images of {len(path_files_list)}')

        return images_not_used
    

In [3]:
get_data = GetCoordsLandmarks()

In [4]:
get_data.create_csv_file('df_dog')

In [6]:
get_data.get_coord('Data\imagenes\perro\*', class_name='dog')

In [7]:
get_data.bad_images()

29 bad images of 602


['Data\\imagenes\\perro\\00000010.png',
 'Data\\imagenes\\perro\\00000035.jpg',
 'Data\\imagenes\\perro\\00000042.jpg',
 'Data\\imagenes\\perro\\00000080.jpg',
 'Data\\imagenes\\perro\\00000120.jpg',
 'Data\\imagenes\\perro\\00000135.jpg',
 'Data\\imagenes\\perro\\00000147.jpg',
 'Data\\imagenes\\perro\\00000168.jpg',
 'Data\\imagenes\\perro\\00000173.jpg',
 'Data\\imagenes\\perro\\00000188.png',
 'Data\\imagenes\\perro\\00000212.png',
 'Data\\imagenes\\perro\\00000220.jpg',
 'Data\\imagenes\\perro\\00000221.jpg',
 'Data\\imagenes\\perro\\00000223.jpg',
 'Data\\imagenes\\perro\\00000224.jpg',
 'Data\\imagenes\\perro\\00000232.jpg',
 'Data\\imagenes\\perro\\00000234.jpg',
 'Data\\imagenes\\perro\\00000235.jpg',
 'Data\\imagenes\\perro\\00000239.jpg',
 'Data\\imagenes\\perro\\00000270.jpg',
 'Data\\imagenes\\perro\\00000281.jpg',
 'Data\\imagenes\\perro\\00000298.jpg',
 'Data\\imagenes\\perro\\00000304.jpg',
 'Data\\imagenes\\perro\\00000315.jpg',
 'Data\\imagenes\\perro\\00000322.jpg',


In [8]:
get_data.get_coord('Data\Videos\perro\*', class_name='perro', file_name='df_dog.csv')